<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalación de las bibliotecas necesarias
!pip install -q transformers umap-learn matplotlib

import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import umap
from transformers import AutoTokenizer, AutoModel

# Montar Google Drive para acceder a los archivos de código
from google.colab import drive
drive.mount('/content/drive')

# Ruta a la carpeta que contiene los archivos de código
code_folder = '/content/drive/MyDrive/CarpetaDeCodigo'  # Reemplaza con la ruta adecuada

# Cargar el modelo y el tokenizador de embeddings
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def read_code_files(folder_path):
    """Lee todos los archivos de código en la carpeta especificada."""
    code_files = glob.glob(os.path.join(folder_path, '*.py'))  # Ajusta la extensión según el lenguaje
    code_snippets = []
    for file_path in code_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                code = file.read()
                code_snippets.append((os.path.basename(file_path), code))
        except Exception as e:
            print(f"Error al leer {file_path}: {e}")
    return code_snippets

def generate_embeddings(texts):
    """Genera embeddings para una lista de textos utilizando el modelo especificado."""
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

# Leer los archivos de código
code_snippets = read_code_files(code_folder)
file_names = [name for name, _ in code_snippets]
code_texts = [code for _, code in code_snippets]

# Generar embeddings
embeddings = generate_embeddings(code_texts)

# Reducir la dimensionalidad para visualización
reducer = umap.UMAP(n_neighbors=5, min_dist=0.3, metric='cosine')
embedding_2d = reducer.fit_transform(embeddings)

# Visualizar los embeddings
plt.figure(figsize=(10, 8))
plt.scatter(embedding_2d[:, 0], embedding_2d[:, 1], s=100, alpha=0.7)
for i, file_name in enumerate(file_names):
    plt.text(embedding_2d[i, 0], embedding_2d[i, 1], file_name, fontsize=9)
plt.title('Visualización de Embeddings de Archivos de Código')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
plt.grid(True)
plt.show()